In [1]:
import pulp 
import heapq
import folium
import math

import networkx as nx
import pandas as pd

from geopy.distance import geodesic

# Imports

In [2]:
df_source = pd.read_csv('/Users/samuele/Desktop/我/CC/FoliaStream/test_dikstra/source.csv')
df_sink = pd.read_csv('/Users/samuele/Desktop/我/CC/FoliaStream/test_dikstra/sink.csv')

In [3]:
df_source['id'] = df_source['id'].astype(int)
df_sink['id'] = df_sink['id'].astype(int)
df_source = df_source.set_index(df_source['id'])
df_sink = df_sink.set_index(df_sink['id'])

In [4]:
# Inputs

transport_cost_0_50000 = 2.5
transport_cost_50000_100000 = 1.5
transport_cost_100000_250000 = 0.9
transport_cost_250000_500000 = 0.6
transport_cost_500000_1000000 = 0.4
transport_cost_1000000_2000000 = 0.3
transport_cost_2000000_999999999 = 0.2

emission_cost = 100

lat_max = max(max(df_sink['latitude']),max(df_source['lat']))
lon_max = max(max(df_sink['longitude']),max(df_source['lon']))
lat_min = min(min(df_sink['latitude']),min(df_source['lat']))
lon_min = min(min(df_sink['longitude']),min(df_source['lon']))

# Support functions

In [5]:
def create_fully_connected_graph(df_source, df_sink):

    # Init graph
    G = nx.DiGraph()

    # Add source nodes
    for idx, row in df_source.iterrows():
        node_id = f"source_{idx}"
        G.add_node(node_id,
                   pos=(row['lat'], row['lon']),
                   type='source')
    
    # Add sink nodes
    for idx, row in df_sink.iterrows():
        node_id = f"sink_{idx}"
        G.add_node(node_id,
                   pos=(row['longitude'], row['latitude']),
                   type='sink')
    
    # Create fully connected graph with geodesic distances 
    for i, (i_id, i_data) in enumerate(G.nodes(data=True)):
        i_lat = df_source.loc[int(i_id.split('_')[1])]['lat'] if 'source' in i_id else df_sink.loc[int(i_id.split('_')[1])]['latitude']
        i_lon = df_source.loc[int(i_id.split('_')[1])]['lon'] if 'source' in i_id else df_sink.loc[int(i_id.split('_')[1])]['longitude']

        for j,(j_id, j_data) in enumerate(G.nodes(data=True)):
            if i_id != j_id: # Avoid self loops
                j_pos = j_data['pos']
                j_lat = df_source.loc[int(j_id.split('_')[1])]['lat'] if 'source' in j_id else df_sink.loc[int(j_id.split('_')[1])]['latitude']
                j_lon = df_source.loc[int(j_id.split('_')[1])]['lon'] if 'source' in j_id else df_sink.loc[int(j_id.split('_')[1])]['longitude']

                # Calculate geodesic distance
                weight = geodesic((i_lat, i_lon), (j_lat, j_lon)).kilometers
                G.add_edge(i_id, j_id, weight=weight)
    return G

In [6]:
def path_dependent_dijkstra(G, source, target):

    # Track distance, paths, and hop counts
    distances = {node: float('infinity') for node in G.nodes()}
    distances[source] = 0
    previous_nodes = {node: None for node in G.nodes()}
    hop_counts = {node:0 for node in G.nodes()}
    hop_counts[source] = 1 # start with 1 node in the path

    # Priority queue for nodes to visti: (adjusted_distance, node)
    priority_queue = [(0,source)]

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Stop if reach target
        if current_node == target:
            break
    
        # Skip if already found better path
        if current_distance > distances[current_node]:
            continue

        # Check all neighbors
        for neighbor in G.neighbors(current_node):

            # Raw edge weight
            edge_weight = G[current_node][neighbor]['weight']

            # Adjust weight based on hop count (divide by hop count)
            adjusted_weight = edge_weight / hop_counts[current_node]

            # Calculate new distances
            distance = distances[current_node] + adjusted_weight

            # Update if better path found
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                hop_counts[neighbor] = hop_counts[current_node] + 1
                heapq.heappush(priority_queue, (distance, neighbor))
    
    # Reconstruct path
    path = []
    current_node = target
    while current_node is not None:
        path.insert(0, current_node)
        current_node = previous_nodes[current_node]

    # Calculate actual distance (not adjusted) for the found path 
    actual_distance = 0 
    for i in range(len(path)-1):
        actual_distance += G[path[i]][path[i+1]]['weight']
    
    return path, distances[target], actual_distance, hop_counts[target]

# Step 1. Generate Dijkstra paths for all source-sink pairs

In [7]:
def generate_all_paths(df_source, df_sink, graph):

    path_registry = {}

    for i, rsource in df_source.iterrows():
        source_node = f'source_{i}'
        for j, rsink in df_sink.iterrows():
            sink_node = f'sink_{j}'

            # Get path using path-dependent Dijkstra
            path, adjusted_distance, actual_distance, hop_count = path_dependent_dijkstra(graph, source_node, sink_node)

            # Register path with its details
            path_registry[(i,j)] = {
                'path':path,
                'adjusted_distance':adjusted_distance,
                'actual_distance':actual_distance,
                'hop_count':hop_count,
                'source_id':f'source_id_{i}',
                'sink_id':f'sink_id_{j}'
            }
            
    return path_registry

# Step 2. Create MCF model with path based flow variables

In [8]:
def create_path_based_mcf_model(df_source, df_sink, path_registry):

    # Init problem
    prob = pulp.LpProblem("PathBasedMCF", pulp.LpMinimize)

    # Create flow variables for each path 
    path_vars = {}
    for (i,j), path_info in path_registry.items():
        var_name = f'Flow_from_{i}_to_{j}'
        path_vars[(i,j)] = pulp.LpVariable(var_name, 0, None, pulp.LpContinuous)

    # Create flow variables for emissions to atmosphere
    atmo_vars = {}
    for i,row in df_source.iterrows():
        var_name = f"Flow_from_{i}_to_atmo"
        atmo_vars[i] = pulp.LpVariable(var_name, 0, None, pulp.LpContinuous)

    # Cost function: placewise linear segments for path-based transportation costs
    path_segment_vars = {}

    for (i,j), path_info in path_registry.items():
        segments = []
        # The scaled distance cost uses actual_distance froim the path
        distance = path_info['actual_distance']

        # Cost segments using path distnace
        cost_segments = [
            (0,50000, transport_cost_0_50000 * distance),
            (50000,100000, transport_cost_50000_100000 * distance),
            (100000,250000, transport_cost_100000_250000 * distance),
            (250000,500000, transport_cost_250000_500000 * distance),
            (500000,1000000, transport_cost_500000_1000000 * distance),
            (1000000,2000000, transport_cost_1000000_2000000 * distance),
            (2000000,999999999, transport_cost_2000000_999999999 * distance),
        ]

        for i_seg, (start, end, slope) in enumerate(cost_segments):
            var = pulp.LpVariable(f'Segment_path_{i}_{j}_{i_seg}', 0, end-start, pulp.LpContinuous)
            segments.append((var, start, end, slope))
        
        path_segment_vars[(i,j)] = segments

    # Same for atmosphere emission costs
    atmo_segment_vars = {}
    for i,row in df_source.iterrows():
        var = pulp.LpVariable(f'Segment_atmo_{i}', 0, 999999999999999999999999999, pulp.LpContinuous)
        atmo_segment_vars[i] = [(var, 0, 999999999999999999999999999, emission_cost)]

    # Objective: Minimize total cost acrtoss all paths and emissions

    prob += pulp.lpSum([var * slope for (i,j) in path_registry for var, start, end, slope in path_segment_vars[(i,j)]] + 
                       [var * slope for i in df_source.index for var, start, end, slope in atmo_segment_vars[i]]), "TotalCost"
    
    # Constraint: all source emission must be extracted from source
    print(atmo_vars)
    for i, rsource in df_source.iterrows():
        prob += (pulp.lpSum([path_vars[(i,j)] for j in df_sink.index if (i,j) in path_vars]) + 
                 atmo_vars[i] == rsource['emission']), f'source_id_{i}_flow_conservation'
        
    # Constraint: link flow variables to their piecewise segments 
    for (i,j) in path_registry:
        prob += path_vars[(i,j)] == pulp.lpSum([var for var, start, end, slope in path_segment_vars[(i,j)]]), f"Piecewise_path_{i}_{j}"
    
    for i, row in df_source.iterrows():
        prob += atmo_vars[i] == pulp.lpSum([var for var, start, end, slope, in atmo_segment_vars[i]]), f'Piecewise_atmo_{i}'
    
    # Optional: capacity constraints for paths
    for (i,j) in path_registry:
        prob += path_vars[(i,j)] <= 10000000000 #000000
    
    return prob, path_vars, atmo_vars

# Step 3. Visualize on map

In [9]:
def visualize_flow_map(map_obj, df_source, df_sink, path_registry, path_vars):

    # Dictionary to track cumulative flow on each edge
    edge_flows = {}

    # First calculate cumulative flow on each edge
    for (i,j), path_info in path_registry.items():
        flow_amount = path_vars[(i,j)].varValue

        if flow_amount > 0:
            path = path_info['path']

            # Process each edge in the path
            for k in range(len(path) - 1):
                edge = (path[k], path[k+1])

                # Add flow to this edge
                if edge in edge_flows:
                    edge_flows[edge] += flow_amount
                else:
                    edge_flows[edge] = flow_amount
    
    # Visualize each edge with its cumulative flow
    for edge, flow in edge_flows.items():
        start_node, end_node = edge

        # Get coordinates for start node
        if "source" in start_node:
            idx = int(start_node.split('_')[1])
            start_lat = df_source.loc[idx]['lat'] ####
            start_lon = df_source.loc[idx]['lon'] ####
        else:
            idx = int(end_node.split('_')[1])
            start_lat = df_sink.loc[idx]['latitude']
            start_lon = df_sink.loc[idx]['longitude']
        
        # Get coordinates for end node
        if "source" in end_node:
            idx = int(end_node.split('_')[1])
            end_lat = df_source.loc[idx]['lat']
            end_lon = df_source.loc[idx]['lon']
        else:
            idx = int(end_node.split('_')[1])
            end_lat = df_sink.loc[idx]['latitude']
            end_lon = df_sink.loc[idx]['longitude']
        
        # Calculate edge weight with cumulative flow tooltip
        weight = 2 + (flow / 1000000) # Adjust as needed

        folium.PolyLine(
            locations=[(start_lat, start_lon), (end_lat, end_lon)],
            weight=weight,
            color='green',
            opacity=0.7,
            tooltip=f"Flow: {flow:.2f} units \n{start_node} ==> {end_node}"
        ).add_to(map_obj)
    
    # Add detail information as a control panel
    html_content = """
    <div style="position: fixed;
                bottom: 50px; right: 50px; width: 280px;
                border:2px solid grey; z-index:9999; font-size:12px;
                background-color:white; padding:10px;
                opacity:0.8;">
    <h4>Edge Details</h4>
    <table>
        <tr><th>Edge</th><th>Flow</th></tr>
    """

    # Sort edges by flow amount for better readability
    sorted_edges = sorted(edge_flows.items(), key=lambda x: x[1], reverse=True)

    for edge, flow in sorted_edges:
        start, end = edge
        html_content += f"<tr><td>{start} ==> {end}</td><td>{flow:.2f}</td></tr>"

    html_content += """
    </table>
    </div>
    """
        
    # Add the HTML content to the map
    folium.Element(html_content).add_to(map_obj)

    return map_obj

# Execute

In [10]:
def main():
    
    # Create graph
    graph = create_fully_connected_graph(df_source, df_sink)

    # Generate all paths
    path_registry = generate_all_paths(df_source, df_sink, graph)

    # Create and solve the MCF model
    prob, path_vars, atmo_vars = create_path_based_mcf_model(df_source, df_sink, path_registry)
    prob.solve()

    # Check solution status
    status = pulp.LpStatus[prob.status]
    print("Status: ", status)

    if status == 'Infeasible':
        print("The model is infeasible.")
    else:
        # Print results
        print("\n Flow Results:")
        for (i,j) in path_registry:
            flow = path_vars[(i,j)].varValue
            if flow > 0:
                path = path_registry[(i,j)]['path']
                print(f"Flow from source_{i} to sink_{j}: {flow:.2f} tons")
                print(f"Path: {'==>'.join(path)}")
                print(f"Distance: {path_registry[(i,j)]['actual_distance']:.2f} km")
                print()

        # Print atmospheric emissions
        for i, row in df_source.iterrows():
            flow = atmo_vars[i].varValue
            if flow>0:
                print(f"Emissions from source_{i} to atmosphere: {flow:.2f} units")

        print("\n Total Cost:", pulp.value(prob.objective))

        # Visualize results
        map = folium.Map(location=((lat_max + lat_min)/2, (lon_max + lon_min)/2), zoom_start=6)

        # Add markers for sources and sinks
        for i, row in df_source.iterrows():
            folium.Marker(
                location=(row['lat'], row['lon']),
                icon=folium.Icon(color='blue'),
                tooltip=f"source_{int(row['id'])}"
            ).add_to(map)
        
        for i, row in df_sink.iterrows():
            folium.Marker(
                location=(row['latitude'], row['longitude']),
                icon=folium.Icon(color='red'),
                tooltip=f"sink_{int(row['id'])}"
            ).add_to(map)

        # Add flow paths
        map = visualize_flow_map(map, df_source, df_sink, path_registry, path_vars)

        # Save or display the map 
        map.save("/Users/samuele/Desktop/我/CC/FoliaStream/test_dikstra/map.html")
        
if __name__ == "__main__":
    main()

{447843: Flow_from_447843_to_atmo, 25450642: Flow_from_25450642_to_atmo, 25451328: Flow_from_25451328_to_atmo, 448783: Flow_from_448783_to_atmo, 449710: Flow_from_449710_to_atmo, 25450244: Flow_from_25450244_to_atmo, 447453: Flow_from_447453_to_atmo, 447456: Flow_from_447456_to_atmo, 25450561: Flow_from_25450561_to_atmo, 447762: Flow_from_447762_to_atmo, 25450608: Flow_from_25450608_to_atmo, 25452738: Flow_from_25452738_to_atmo, 25452723: Flow_from_25452723_to_atmo, 25452187: Flow_from_25452187_to_atmo, 25452991: Flow_from_25452991_to_atmo, 25453551: Flow_from_25453551_to_atmo, 25452183: Flow_from_25452183_to_atmo, 447847: Flow_from_447847_to_atmo, 25453669: Flow_from_25453669_to_atmo, 25453589: Flow_from_25453589_to_atmo, 449699: Flow_from_449699_to_atmo, 25450666: Flow_from_25450666_to_atmo, 25454624: Flow_from_25454624_to_atmo, 449388: Flow_from_449388_to_atmo, 25454799: Flow_from_25454799_to_atmo, 717512: Flow_from_717512_to_atmo, 25454798: Flow_from_25454798_to_atmo, 450208: Flow_

# Additional functions

In [11]:
# Custom dijkstra implementation that avoids specified node types
def path_dependent_dijkstra_with_exclusions(G, source, target, exclude_types=None):
    
    if exclude_types is None:
        exclude_types = []
    
    # Track distances, paths, and hop counts
    distances = {node: float('infinity') for node in G.nodes()}
    distances[source] = 0
    previous_nodes = {node: None for node in G.nodes()}
    hop_counts = {node: 0 for node in G.nodes()}
    hop_counts[source] = 1

    # Priority queue for nodes to visit: (adjusted_distance, node)
    priority_queue = [(0, source)]

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Stop if we reached the target
        if current_node == target:
            break
        
        # Skip if already found better path
        if current_distance > distances[current_node]:
            continue

        # Check all neighbors
        for neighbor in G.neighbors(current_node):
            # Skip excluded node types (except target)
            if neighbor != target and G.nodes[neighbor].get('type') in exclude_types:
                continue
            
            # Raw edge weight
            edge_weight = G[current_node][neighbor]['weight']

            # Adjust weight based on hop count (divide by hop count)
            adjusted_weight = edge_weight / hop_counts[current_node]

            # Calculate new distance
            distance = distances[current_node] + adjusted_weight

            # Update if better path found 
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                hop_counts[neighbor] = hop_counts[current_node] + 1
                heapq.heappop(priority_queue, (distance, neighbor))

    # Reconstruct path 
    path = []
    current_node = target
    while current_node is not None:
        path.insert(0, current_node)
        current_node = previous_nodes[current_node]
    
    # Check if path exists
    if path[0] != source:
        return [], float('infinity'), float('infinity'), 0
    
    # Calculate actual distance (not adjusted) for the found path
    actual_distance = 0
    for i in range(len(path) - 1):
        actual_distance += G[path[i]][path[i+1]]['weight']
    
    return path, distances[target], actual_distance, hop_counts[target]

In [12]:
# Find path that must go through specific waypoints without using sink nodes in intermediate segments

def path_with_required_waypoints(G, source, target, waypoints_df):

    # Get waypoints that must be visited
    source_waypoints = waypoints_df[waypoints_df['id'] == source]['waypoint_id'].to_list()

    if not source_waypoints:
        # No required waypoints --> find direct path
        return path_dependent_dijkstra(G, source, target)
    
    # Add target as the final destination
    ordered_stops = source_waypoints + [target]

    # Initialize
    complete_path = []
    total_adjusted_distance = 0
    total_actual_distance = 0
    current = source
    hop_count = 1

    # Find path segments through each waypoint in sequence
    for i, next_stop in enumerate(ordered_stops):

        # Avoid sink nodes except last one
        if i < len(ordered_stops) - 1:
            segment_path, semgent_adjusted, segment_actual, segment_hops = path_dependent_dijkstra_with_exclusions(G, current, next_stop, exclude_types=['sink'])
        else:
            # Normal dijkstra for last segment (final sink)
            segment_path, semgent_adjusted, segment_actual, segment_hops = path_dependent_dijkstra(G, current, next_stop)

        # Check if a path was found
        if not segment_path:
            print(f"No valit path found from {current} to {next_stop} with given constraints")
            return [], 0, 0, 0

        # Add segment to complete path (avoid duplicates)
        if complete_path:
            complete_path.extend(segment_path[1:]) # Skip first node to avoid duplication
        else:
            complete_path = segment_path

        # Update metrics
        total_adjusted_distance += semgent_adjusted
        total_actual_distance += segment_actual
        hop_count = len(complete_path)

        # Update current position
        current = next_stop

    return complete_path, total_adjusted_distance, total_actual_distance, hop_count

In [13]:
def calculate_distance(node1, node2):
    return math.sqrt((node2[0] - node1[0]) ** 2 + (node2[1] - node1[1]) ** 2)

In [14]:
# coordinates = {
#     'Node1':(1,1),
#     'Node2':(2,3),
#     'Node3':(3,5),
#     'Node4':(4,7),
#     'Node5':(5,9),
#     'Node6':(6,11),
#     'Node7':(7,13),
#     'Node8':(8,15),
#     'Node9':(9,17),
#     'Node10':(10,19),
#     'SinkNode1':(15,15),
#     'SinkNode2':(20,20),
# }

# graph = {node: {} for node in coordinates}

# for node1 in coordinates:
#     for node2 in coordinates:
#         if node1 != node2:
#             graph[node1][node2] = calculate_distance(coordinates[node1], coordinates[node2])

# waypoints_df = pd.DataFrame([
#     {'id':447843, 'waypoint_id':25450642}
# ])

# path, adjusted_dist, actual_dist, hops = path_with_required_waypoints(graph, 1, 1, waypoints_df)